#### Introduction.
In this tutorial , we will simulate PID Controllers to see set point and disturbance  response to a first order system, how to troubleshoot the PID controllers , and how to obtain the optimal PID paramters 
PID ( proportioal integral and Derivative controllers )  controllers are the workhorses of a process control system in a plant. 
The algorithm of PID controllers in interactive form is given by 


$$ MV(t) = k_p( E(t) + \frac{1}{T_i}\int _{0}^{t}E(t)dt+ T_d E(t) ) $$

where $k_p$ is the proportional gain , $T_i$ is the integral time and $T_d$ is the derivative gain.

In transfer function form this is represented as 

 $$ K_p (1+ \frac{1}{T_i}s + T_ds) $$

In [1]:
import scipy.signal
import numpy
import matplotlib.pyplot as plt
%matplotlib inline
import sympy

In [2]:
class PIController:
    def __init__(self, Kc, tau_i, bias):
        self.G = scipy.signal.lti([Kc*tau_i, Kc], [tau_i, 0]) # model of PID transfer function
        self.change_state(numpy.zeros((self.G.A.shape[0], 1))) # 18 is the initial PV
        self.bias = self.output = bias
        self.y = self.bias
        
    def change_input(self, u):
        self.y = self.G.C.dot(self.x) + self.G.D.dot(u) + self.bias
        self.output = self.y[0, 0]  # because y is a matrix, and we want a scalar output
    
    def change_state(self, x):
        self.x = self.state = x
    
    def derivative(self, e):
        return self.G.A.dot(self.x) + self.G.B.dot(e)

In [3]:
import numpy
ts = numpy.linspace(0,100,1000)
dt= ts[1]
SV = 50


In [4]:
class LtiSystem:
    def __init__(self,num,den):
        self.G =scipy.signal.lti(num,den)
        self.change_state(numpy.ones((self.G.A.shape[0],1))) # initialize the PV
        self.y = self.output = 40
    
    def change_input(self,u):
        self.y = self.G.C.dot(self.x)
        self.output = self.y[0,0]
        
    def change_state(self,x):
        self.x = self.state = x
        
    def derivative(self,e):
        return self.G.A.dot(self.x) + self.G.B.dot(e)
    
    

In [5]:
def control_simulation(system, controller):
    outputs = []
    for t in ts:
        system.change_input(controller.output)

        e = sp - system.output

        controller.change_input(e)

        system.change_state(system.state + system.derivative(controller.output)*dt)
        controller.change_state(controller.state + controller.derivative(e)*dt)

        outputs.append(system.output)
    return outputs
        
        

In [6]:
ts = numpy.linspace(0, 100, 100)
dt = ts[1]
sp = 20

In [7]:

def simulate(kc, kp, ti , t):
    outputs = control_simulation(system=LtiSystem(kp, [t, 1]), 
                             controller=PIController(kc, tau_i=ti, bias=0))
   
    plt.plot(ts,  outputs)
    plt.plot([sp]*100)
    


In [8]:
from ipywidgets import interact

In [9]:
interact(simulate, kc=(0.1,10,0.1),kp=(0.1,10,0.1),ti=(0.1,10,0.1),t=(0.1,10,0.1))

interactive(children=(FloatSlider(value=5.0, description='kc', max=10.0, min=0.1), FloatSlider(value=5.0, description='kp', max=10.0, min=0.1), FloatSlider(value=5.0, description='ti', max=10.0, min=0.1), FloatSlider(value=5.0, description='t', max=10.0, min=0.1), Output()), _dom_classes=('widget-interact',))

<function __main__.simulate>

In [10]:
from ipywidgets import ToggleButtons

In [11]:
toggle = ToggleButtons()